---

# Yandex.Music

# Contents <a id='back'></a>

* [Introducción](#intro)
* [Etapa 1. Descripción de los datos](#data_review)
    * [Conclusions](#data_review_conclusions)
* [Etapa 2. Data preprocessing](#data_preprocessing)
    * [2.1 Estilo del encabezado](#header_style)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Duplicados](#duplicates)
    * [2.4 Conclusiones](#data_preprocessing_conclusions)
* [Etapa 3. Prueba de hipótesis](#hypotheses)
    * [3.1 Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades](#activity)
    * [3.2 Hipótesis 2: música al principio y al final de la semana](#week)
    * [3.3 Hipótesis 3: preferencias de género en Springfield y Shelbyville](#genre)
* [Conclusiones](#end)

## Introducción <a id='intro'></a>
Siempre que investiguemos, necesitamos formular hipótesis que después podamos probar. A veces aceptamos estas hipótesis; otras, las rechazamos. Para tomar las decisiones correctas, una empresa debe ser capaz de entender si está haciendo las suposiciones correctas.

En este proyecto, compararás las preferencias musicales de las ciudades de Springfield y Shelbyville. Estudiarás datos reales de Yandex.Music para probar las hipótesis de abajo y comparar el comportamiento del usuario de esas dos ciudades.

### Objetivo: 
Prueba tres hipótesis: 
1. La actividad de los usuarios difiere según el día de la semana y dependiendo de la ciudad. 
2. Los lunes por la mañana, los habitantes de Springfield y Shelbyville escuchan diferentes géneros. Lo mismo ocurre con los viernes por la noche. 
3. Los oyentes de Springfield y Shelbyville tienen preferencias distintas. En Springfield prefieren el pop mientras que en Shelbyville hay más aficionados al rap.

### Etapas 
Los datos del comportamiento del usuario se almacenan en el archivo `/datasets/music_project_en.csv`. No hay ninguna información sobre la calidad de los datos así que necesitarás examinarlos antes de probar las hipótesis. 

Primero, evaluarás la calidad de los datos y verás si los problemas son significativos. Entonces, durante el preprocesamiento de datos, tomarás en cuenta los problemas más críticos.
 
Tu proyecto consistirá en tres etapas:
 1. Descripción de los datos
 2. Preprocesamiento de datos
 3. Prueba de hipótesis
 


## Etapa 1. Descripción de los datos <a id='data_review'></a>



In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('/datasets/music_project_en.csv')
df.describe()

,userID,Track,artist,genre,City,time,Day
count,65079,63736,57512,63881,65079,65079,65079
unique,41748,39666,37806,268,2,20392,3
top,A8AE9169,Brand,Kartvelli,pop,Springfield,08:14:07,Friday
freq,76,136,136,8850,45360,14,23149


In [3]:
print(df.head(10))

     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                       Chains          Obladaet  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE                     L’estate       Julia Dalia  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

        City        time        Day  
0  Shelbyville  20:28:33  Wednesday  
1  Springfield  14:07:09     Friday  
2  Shelbyville  20:58:07  Wednesday  
3  Shelbyville  08:37:09     Monday  
4  Springfield  08:34:34     Monday  
5

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


La tabla contiene siete columnas. Todas almacenan el mismo tipo de datos: objeto.

De acuerdo con la documentación:
- `'userID'` — identificador del usuario
- `'Track'` — título de la pista
- `'artist'` — nombre del artista
- `'genre'` — género
- `'City'` — ciudad del usuario
- `'time'` — el periodo de tiempo exacto en que se reprodujo la pista
- `'Day'` — día de la semana

Podemos ver tres problemas con el estilo en los nombres de las columnas:
1. Algunos nombres están en mayúsculas, otros en minúsculas.
2. Hay algunos espacios en algunos nombres.
3. `Detecta el tercer problema tú mismo y descríbelo aquí`.

El número de valores de las columnas es diferente. Esto significa que los datos contienen valores ausentes.


### Conclusiones <a id='data_review_conclusions'></a> 

Cada fila de la tabla almacena datos de la pista que fue reproducida. Algunas columnas describen la pista en sí: su título, el artista y el género. El resto transmite la información del usuario: la ciudad de la que viene, el tiempo que ha reproducido la pista. 

Está claro que los datos son suficientes para probar la hipótesis. Sin embargo, hay valores ausentes.

Para continuar, necesitamos preprocesar los datos.

## Etapa 2. Preprocesamiento de datos <a id='data_preprocessing'></a>


### Estilo del encabezado <a id='header_style'></a>


In [5]:
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


In [6]:
df = df.rename(columns={
    '  userID': 'user_id',
    'Track': 'track',
    '  City  ': 'city',
    'Day': 'day'
})

In [7]:
print(df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


### Valores ausentes <a id='missing_values'></a>


In [8]:
print(df.isna().sum())

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64


No todos los valores ausentes afectan a la investigación. Por ejemplo, los valores ausentes en la pista y artista no son cruciales. Simplemente puedes reemplazarlos por marcadores claros.

Pero los valores ausentes en `'genre'` pueden afectar la comparación entre las preferencias musicales de Springfield y Shelbyville. En la vida real, sería útil saber las razones por las cuales hay datos ausentes e intentar recuperarlos. Pero no tenemos esa oportunidad en este proyecto. Así que tendrás que:
* Rellenar esos valores ausentes con marcadores
* Evaluar cuánto podrían afectar los valores ausentes a tus cómputos.

In [9]:
columns_to_replace = ['track', 'artist', 'genre']

for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')
    


In [10]:
print(df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### Duplicados <a id='duplicates'></a>


In [11]:

print(df.duplicated().sum())

3826


In [12]:
pandas = df.drop_duplicates()
print(pandas)

        user_id                              track            artist  \
0      FFB692EC                  Kamigata To Boots  The Mass Missile   
1      55204538        Delayed Because of Accident  Andreas Rönnberg   
2        20EC38                  Funiculì funiculà       Mario Lanza   
3      A3DD03C9              Dragons in the Sunset        Fire + Ice   
4      E2DC1FAE                        Soul People        Space Echo   
...         ...                                ...               ...   
65074  729CBB09                            My Name            McLean   
65075  D08D4A55  Maybe One Day (feat. Black Spade)       Blu & Exile   
65076  C5E3A0D5                          Jalopiina           unknown   
65077  321D0506                      Freight Train     Chas McDevitt   
65078  3A64EF84          Tell Me Sweet Little Lies      Monica Lopez   

            genre         city      time        day  
0            rock  Shelbyville  20:28:33  Wednesday  
1            rock  Springfi

In [13]:
print(pandas.duplicated().sum())

0


In [14]:
print(sorted(df['genre'].unique()))

['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans', 'alternative', 'ambient', 'americana', 'animated', 'anime', 'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook', 'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber', 'children', 'chill', 'chinese', 'choral', 'christian', 'christmas', 'classical', 'classicmetal', 'club', 'colombian', 'comedy', 'conjazz', 'contemporary', 'country', 'cuban', 'dance', 'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic', 'electropop', 'emo', 'entehno', 'epicmetal', 'estrada', 'ethnic', 'eurofolk', 'european', 'experimental', 'extrememetal', 'fado', 'film', 'fitness', 'flamenco', 'folk', 'folklor

In [15]:
# función para reemplazar duplicados implícitos
def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre)
        


In [16]:
# eliminando duplicados implícitos
duplicates = ['hip', 'hop', 'hip-hop']
genre = 'hiphop'
replace_wrong_genres(duplicates, genre)
print(df)

        user_id                              track            artist  \
0      FFB692EC                  Kamigata To Boots  The Mass Missile   
1      55204538        Delayed Because of Accident  Andreas Rönnberg   
2        20EC38                  Funiculì funiculà       Mario Lanza   
3      A3DD03C9              Dragons in the Sunset        Fire + Ice   
4      E2DC1FAE                        Soul People        Space Echo   
...         ...                                ...               ...   
65074  729CBB09                            My Name            McLean   
65075  D08D4A55  Maybe One Day (feat. Black Spade)       Blu & Exile   
65076  C5E3A0D5                          Jalopiina           unknown   
65077  321D0506                      Freight Train     Chas McDevitt   
65078  3A64EF84          Tell Me Sweet Little Lies      Monica Lopez   

            genre         city      time        day  
0            rock  Shelbyville  20:28:33  Wednesday  
1            rock  Springfi

In [17]:
# revisando en busca de duplicados implícitos
print(sorted(df['genre'].unique()))

['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans', 'alternative', 'ambient', 'americana', 'animated', 'anime', 'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook', 'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber', 'children', 'chill', 'chinese', 'choral', 'christian', 'christmas', 'classical', 'classicmetal', 'club', 'colombian', 'comedy', 'conjazz', 'contemporary', 'country', 'cuban', 'dance', 'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic', 'electropop', 'emo', 'entehno', 'epicmetal', 'estrada', 'ethnic', 'eurofolk', 'european', 'experimental', 'extrememetal', 'fado', 'film', 'fitness', 'flamenco', 'folk', 'folklor

### Conclusiones <a id='data_preprocessing_conclusions'></a>
Detectamos tres problemas con los datos:

- Estilos de encabezados incorrectos
- Valores ausentes
- Duplicados obvios e implícitos

Los encabezados han sido eliminados para conseguir que el procesamiento de la tabla sea más sencillo.

Todos los valores ausentes han sido reemplazados por `'unknown'`. Pero todavía tenemos que ver si los valores ausentes en `'genre'` afectan a nuestros cálculos.

La ausencia de duplicados hará que los resultados sean mas precisos y fáciles de entender.

Ahora ya podemos continuar probando las hipótesis. 

## Etapa 3. Prueba de hipótesis <a id='hypotheses'></a>

### Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades <a id='activity'></a>

De acuerdo con la primera hipótesis, los usuarios de Springfield y Shelbyville escuchan música de forma distinta. Comprueba esto utilizando los datos de tres días de la semana: lunes, miércoles y viernes.

* Divide a los usuarios en grupos por ciudad.
* Compara cuántas pistas reprodujo cada grupo el lunes, el miércoles y el viernes.


In [18]:
# contando las pistas reproducidas en cada ciudad
df.groupby('city')['track'].count()


city
Shelbyville    19719
Springfield    45360
Name: track, dtype: int64

Springfield ha reproducido más pistas que Shelbyville. Pero eso no implica que los ciudadanos de Springfield escuchen música más a menudo. Esta ciudad es simplemente más grande y hay más usuarios.



In [19]:
df.groupby('day')['track'].count()

day
Friday       23149
Monday       22697
Wednesday    19233
Name: track, dtype: int64

El miércoles fue el día más silencioso de todos. Pero si consideramos las dos ciudades por separado podríamos llegar a una conclusión diferente.

In [20]:
# <creando la función number_tracks()>

def number_tracks(day='Monday',city='Springfield'):
    track_list = df[df['day'] == day] 
    track_list = track_list[track_list['city'] == city] 
    track_list_count = track_list['user_id'].count()
    
    return track_list_count
print(number_tracks())

# declararemos la función con dos parámetros: day=, city=.
# deja que la variable track_list almacene las filas df en las que
# el valor en la columna 'day' es igual al parámetro day= y, al mismo tiempo, 
# el valor de la columna 'city' es igual al parámetro city= (aplica el filtrado consecutivo 
# con indexación lógica).
# deja que la variable track_list_count almacene el número de valores de la columna 'user_id' en track_list
# (encontrado con el método count()).
# permite que la función devuelva un número: el valor de track_list_count.

# la función cuenta las pistas reproducidas en un cierto día y ciudad.
# primero recupera las filas del día deseado de la tabla,
# después filtra las filas de la ciudad deseada del resultado,
# entonces, encuentra el número de valores de 'user_id' en la tabla filtrada,
# y devuelve ese número.
# para ver lo que devuelve, envuelve la llamada de la función en print().

16715


In [21]:
# el número de canciones reproducidas en Springfield el lunes
print(number_tracks('Monday', 'Springfield'))

16715


In [22]:
# el número de canciones reproducidas en Shelbyville el lunes
print(number_tracks('Monday', 'Shelbyville'))

5982


In [23]:
# el número de canciones reproducidas en Springfield el miércoles
print(number_tracks('Wednesday', 'Springfield'))

11755


In [24]:
# el número de canciones reproducidas en Shelbyville el miércoles
print(number_tracks('Wednesday', 'Shelbyville'))

7478


In [25]:
# el número de canciones reproducidas en Springfield el viernes
print(number_tracks('Friday', 'Springfield'))

16890


In [26]:
# el número de canciones reproducidas en Shelbyville el viernes
print(number_tracks('Friday', 'Shelbyville'))

6259


Utiliza `pd.DataFrame` para crear una tabla, donde
* Los nombres de las columnas son: `['city', 'monday', 'wednesday', 'friday']`
* Los datos son los resultados que conseguiste de `number_tracks()`

In [27]:
columnas = ['city','monday','wednesday','friday']
datos = [
            ['Springfield', 16715,11755,16890],
            ['Shelbyville', 5982,7478,6259]
        ]

cities_table = pd.DataFrame(data=datos,columns=columnas)

In [28]:
# tabla con los resultados
print(cities_table)

          city  monday  wednesday  friday
0  Springfield   16715      11755   16890
1  Shelbyville    5982       7478    6259


**Conclusiones**

Los datos revelan las diferencias en el comportamiento de los usuarios:

- En Springfield, el número de canciones reproducidas alcanzan el punto máximo los lunes y viernes mientras que los miércoles hay un descenso de la actividad.
- En Shelbyville, al contario, los usuarios escuchan más música los miércoles. La actividad de los usuarios los lunes y viernes es menor.

Así que la primera hipótesis parece ser correcta.

### Hipótesis 2: música al principio y al final de la semana <a id='week'></a>

De acuerdo con la segunda hipótesis, los lunes por la mañana y los viernes por la noche los ciudadanos de Springfield escuchan géneros que difieren de aquellos que los usuarios de Shelbyville disfrutan.

Obtén tablas (asegúrate de que el nombre de tu tabla combinada encaja con el DataFrame dado en los dos bloques de código de abajo):
* Para Springfield — `spr_general`
* Para Shelbyville — `shel_general`

In [29]:
# obteniendo la tabla spr_general de las filas de df, 
# donde los valores en la columna 'city' es 'Springfield'
spr_general = df[df['city'] == 'Springfield']
print(spr_general)

        user_id                          track                   artist  \
1      55204538    Delayed Because of Accident         Andreas Rönnberg   
4      E2DC1FAE                    Soul People               Space Echo   
6      4CB90AA5                           True             Roman Messer   
7      F03E1C1F               Feeling This Way          Polina Griffith   
8      8FA1D3BE                       L’estate              Julia Dalia   
...         ...                            ...                      ...   
65073  83A474E7  I Worship Only What You Bleed  The Black Dahlia Murder   
65074  729CBB09                        My Name                   McLean   
65076  C5E3A0D5                      Jalopiina                  unknown   
65077  321D0506                  Freight Train            Chas McDevitt   
65078  3A64EF84      Tell Me Sweet Little Lies             Monica Lopez   

              genre         city      time        day  
1              rock  Springfield  14:07:09 

In [30]:
# obteniendo shel_general de las filas df,
# donde el valor de la columna 'city' es 'Shelbyville'
shel_general = df[df['city'] == 'Shelbyville']
print(shel_general)

        user_id                              track            artist  \
0      FFB692EC                  Kamigata To Boots  The Mass Missile   
2        20EC38                  Funiculì funiculà       Mario Lanza   
3      A3DD03C9              Dragons in the Sunset        Fire + Ice   
5      842029A1                             Chains          Obladaet   
9      E772D5C0                          Pessimist           unknown   
...         ...                                ...               ...   
65064  BC8EC5CF       Red Lips: Gta (Rover Rework)             Rover   
65065  29E04611                       Bre Petrunko     Perunika Trio   
65066  1B91C621             (Hello) Cloud Mountain   sleepmakeswaves   
65067  F1B93F29                        Poison Kiss       Centerstone   
65075  D08D4A55  Maybe One Day (feat. Black Spade)       Blu & Exile   

            genre         city      time        day  
0            rock  Shelbyville  20:28:33  Wednesday  
2             pop  Shelbyvi

In [31]:
# declarando la función genre_weekday() con los parámetros day=, time1= y time2=. Debería
# devolver información sobre los géneros más populares de un determinado día a una determinada hora:

# 1) Deja que la variable genre_df almacene las filas que cumplen varias condiciones:
#    - el valor de la columna 'day' es igual al valor del argumento day=
#    - el valor de la columna 'time' es mayor que el valor del argumento time1=
#    - el valor en la columna 'time' es menor que el valor del argumento time2=
#    Utiliza un filtrado consecutivo con indexación lógica.

# 2) Agrupa genre_df por la columna 'genre', toma una de sus columnas, 
#    y utiliza el método count() para encontrar el número de entradas por cada uno de 
#    los géneros representados; almacena los Series resultantes en
#    la variable genre_df_count

# 3) Ordena genre_df_count en orden descendente de frecuencia y guarda el resultado
#    en la variable genre_df_sorted

# 4) Devuelve un objeto Series con los primeros 15 valores de genre_df_sorted - los 15
#    géneros más populares (en un determinado día, en un determinado periodo de tiempo)

# Escribe tu función aquí
def genre_weekday(df,day,time_1,time_2):
    # filtrado consecutivo
    # genre_df solo almacenará aquellas filas df en las que el día sea igual a day=
    genre_df = df[df['day']==day] # escribe tu código aquí

    # genre_df solo almacenará aquellas filas df en las que el tiempo sea menos que time2=
    genre_df = genre_df[genre_df['time']<time_2] # escribe tu código aquí
    # genre_df solo almacenará aquellas filas df en las que el tiempo sea mayor que time1=
    genre_df = genre_df[genre_df['time']>time_1] # escribe tu código aquí

    # agrupa el DataFrame filtrado por la columna con los nombres de los géneros, toma la columna de género, y encuentra el número de filas por cada género con el método count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count() # escribe tu código aquí

    # ordenaremos el resultado en orden descendente (por lo que los géneros más populares aparecerán primero en el objeto Series)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)# escribe tu código aquí

    # devolveremos el objeto Series que almacena los 15 géneros más populares en un día determinado en un periodo de tiempo determinado
    return genre_df_sorted[:15]

In [32]:
# llamando a la función para el lunes por la mañana en Springfield (utilizando spr_general en vez de la tabla df)
print(genre_weekday(spr_general,'Monday','07:00','11:00'))

genre
pop            830
dance          589
rock           511
electronic     501
hiphop         311
ruspop         203
world          190
rusrap         188
alternative    175
unknown        172
classical      167
metal          126
jazz           109
folk           107
soundtrack      97
Name: genre, dtype: int64


In [33]:
# llamando a la función para el lunes por la mañana en Shelbyville (utilizando shel_general en vez de la tabla df)
print(genre_weekday(shel_general,'Monday','07:00','11:00'))

genre
pop            238
dance          192
rock           173
electronic     154
hiphop          88
ruspop          68
alternative     65
rusrap          56
jazz            47
classical       42
world           39
soundtrack      34
rap             33
rnb             31
metal           28
Name: genre, dtype: int64


In [34]:
# llamando a la función para el viernes por la tarde en Springfield
print(genre_weekday(spr_general,'Friday','17:00','23:00'))


genre
pop            761
rock           546
dance          521
electronic     510
hiphop         282
world          220
ruspop         184
alternative    176
classical      171
rusrap         151
jazz           121
unknown        117
soundtrack     112
metal           92
rnb             92
Name: genre, dtype: int64


In [35]:
# llamando a la función para el viernes por la tarde en Shelbyville
print(genre_weekday(shel_general,'Friday','17:00','23:00'))


genre
pop            279
rock           230
electronic     227
dance          221
hiphop         103
alternative     67
rusrap          66
jazz            66
classical       64
world           60
unknown         49
ruspop          49
soundtrack      40
rap             39
metal           39
Name: genre, dtype: int64


**Conclusión**

Habiendo comparado los 15 géneros más populares del lunes por la mañana podemos concluir lo siguiente:

1. Los usuarios de Springfield y Shelbyville escuchan música similar. Los cinco géneros más populares son los mismos, solo rock y electrónica han intercambiado posiciones.

2. En Springfield el número de valores ausentes resultaron ser tan altos que el valor `'unknown'` llegó al décimo. Esto significa que los valores ausentes forman una parte considerable de los datos, lo que podría ser la base de la cuestión sobre la fiabilidad de nuestras conclusiones.

Para el viernes por la tarde, la situación es similar. Los géneros individuales varían algo pero, en general, los 15 más populares son parecidos en las dos ciudades.

De esta forma, la segunda hipótesis ha sido parcialmente demostrada:
* Los usuarios escuchan música similar al principio y al final de la semana.
* No hay una gran diferencia entre Springfield y Shelbyville. En ambas ciudades, el world es el género más popular.

Sin embargo, el número de valores ausentes hace este resultado un tanto cuestionable. En Springfield, hay tantos que afectan a nuestros 15 más populares. De no faltarnos esos valores, las cosas podrían parecer diferentes.

### Hipótesis 3: preferencias de género en Springfield y Shelbyville <a id='genre'></a>

Hipótesis: Shelbyville ama la música rap. A los ciudadanos de Springfield les gusta más el pop.

In [36]:
# en una línea: agrupa la tabla spr_general por la columna 'genre', 
# cuenta los valores 'genre' con count() en la agrupación, 
# ordena el Series resultante en orden descendiente, y almacénalo en spr_genres
spr_genres = spr_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [37]:
# imprimiendo las 10 primeras filas de spr_genres
print(spr_genres.head(10))

genre
pop            6253
dance          4707
rock           4188
electronic     4010
hiphop         2215
classical      1712
world          1516
alternative    1466
ruspop         1453
rusrap         1239
Name: genre, dtype: int64


In [38]:
# en una línea: agrupa la tabla shel_general por la columna 'genre', 
# cuenta los valores 'genre' en el agrupamiento con count(), 
# ordena el Series resultante en orden descendente y guárdalo en shel_genres
shel_genres = shel_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [39]:
# imprimiendo las 10 primeras filas de shel_genres
print(shel_genres.head(10))

genre
pop            2597
dance          2054
rock           2004
electronic     1842
hiphop         1020
alternative     700
classical       684
rusrap          604
ruspop          565
world           553
Name: genre, dtype: int64


**Conclusión**

La hipótesis ha sido parcialmente demostrada:
* La música pop es el género más popular en Springfield, tal como se esperaba.
* Sin embargo, la música pop ha resultado ser igual de popular en Springfield que en Shelbyville y el rap no estaba entre los 5 más populares en ninguna de las ciudades.


# Conclusiones <a id='end'></a>

Hemos probado las siguientes tres hipótesis:

1. La actividad de los usuarios difiere dependiendo del día de la semana y de las distintas ciudades. 
2. Los lunes por la mañana los residentes de Springfield y Shelbyville escuchan géneros distintos. Lo mismo ocurre con los viernes por la noche.
3. Los oyentes de Springfield y Shelbyville tienen distintas preferencias. En ambas ciudades, Springfield y Shelbyville, se prefiere el pop.

Tras analizar los datos, concluimos:

1. La actividad del usuario en Springfield y Shelbyville depende del día de la semana aunque las ciudades varían de diferentes formas. 

La primera hipótesis ha sido aceptada completamente.

2. Las preferencias musicales no varían significativamente en el transcurso de la semana en Springfield y Shelbyville. Podemos observar pequeñas diferencias en el orden los lunes, pero:
* En Springfield y Shelbyville la gente lo que más escucha es la música pop.

Así que no podemos aceptar esta hipótesis. También debemos tener en cuenta que el resultado podría haber sido diferente si no fuera por los valores ausentes.

3. Resulta que las preferencias musicales de los usuarios de Springfield y Shelbyville son bastante parecidas.

La tercera hipótesis es rechazada. Si hay alguna diferencia en las preferencias no se puede observar en los datos.

